#Spider.py
The first program (spider.py) program crawls a web site and pulls a series of pages into the database (spider.sqlite), recording the links between pages. You can restart the process at any time by removing the spider.sqlite file and rerunning spider.py.

In [8]:
import sqlite3
import urllib.error
import ssl
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('/content/drive/My Drive/Coursera Learning (OWN)/Python Retrieving Processing Visualizing Data/spider.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages
    (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT,
     error INTEGER, old_rank REAL, new_rank REAL)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Links
    (from_id INTEGER, to_id INTEGER)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

# Check to see if we are already in progress...
cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()
if row is not None:
    print("Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.")
else :
    starturl = input('Enter web url or enter: ')
    if ( len(starturl) < 1 ) : starturl = 'http://seu.edu.bd/'
    if ( starturl.endswith('/') ) : starturl = starturl[:-1]
    web = starturl
    if ( starturl.endswith('.htm') or starturl.endswith('.html') ) :
        pos = starturl.rfind('/')
        web = starturl[:pos]

    if ( len(web) > 1 ) :
        cur.execute('INSERT OR IGNORE INTO Webs (url) VALUES ( ? )', ( web, ) )
        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( starturl, ) )
        conn.commit()

# Get the current webs
cur.execute('''SELECT url FROM Webs''')
webs = list()
for row in cur:
    webs.append(str(row[0]))

print(webs)

many = 0
while True:
    if ( many < 1 ) :
        sval = input('How many pages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)
    many = many - 1

    cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
    try:
        row = cur.fetchone()
        # print row
        fromid = row[0]
        url = row[1]
    except:
        print('No unretrieved HTML pages found')
        many = 0
        break

    print(fromid, url, end=' ')

    # If we are retrieving this page, there should be no links from it
    cur.execute('DELETE from Links WHERE from_id=?', (fromid, ) )
    try:
        document = urlopen(url, context=ctx)

        html = document.read()
        if document.getcode() != 200 :
            print("Error on page: ",document.getcode())
            cur.execute('UPDATE Pages SET error=? WHERE url=?', (document.getcode(), url) )

        if 'text/html' != document.info().get_content_type() :
            print("Ignore non text/html page")
            cur.execute('DELETE FROM Pages WHERE url=?', ( url, ) )
            conn.commit()
            continue

        print('('+str(len(html))+')', end=' ')

        soup = BeautifulSoup(html, "html.parser")
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except:
        print("Unable to retrieve or parse page")
        cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
        conn.commit()
        continue

    cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( url, ) )
    cur.execute('UPDATE Pages SET html=? WHERE url=?', (memoryview(html), url ) )
    conn.commit()

    # Retrieve all of the anchor tags
    tags = soup('a')
    count = 0
    for tag in tags:
        href = tag.get('href', None)
        if ( href is None ) : continue
        # Resolve relative references like href="/contact"
        up = urlparse(href)
        if ( len(up.scheme) < 1 ) :
            href = urljoin(url, href)
        ipos = href.find('#')
        if ( ipos > 1 ) : href = href[:ipos]
        if ( href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif') ) : continue
        if ( href.endswith('/') ) : href = href[:-1]
        # print href
        if ( len(href) < 1 ) : continue

		# Check if the URL is in any of the webs
        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                found = True
                break
        if not found : continue

        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( href, ) )
        count = count + 1
        conn.commit()

        cur.execute('SELECT id FROM Pages WHERE url=? LIMIT 1', ( href, ))
        try:
            row = cur.fetchone()
            toid = row[0]
        except:
            print('Could not retrieve id')
            continue
        # print fromid, toid
        cur.execute('INSERT OR IGNORE INTO Links (from_id, to_id) VALUES ( ?, ? )', ( fromid, toid ) )


    print(count)

conn.close()


Enter web url or enter: http://seu.edu.bd/
['http://seu.edu.bd']
How many pages:100
1 http://seu.edu.bd (77103) 109
66 http://seu.edu.bd/downloads/7th_Convocation_Students_List.pdf Ignore non text/html page
20 http://seu.edu.bd/admission.php?id=scholarship_waiver (17951) 67
40 http://seu.edu.bd/research.php?id=journals (16711) 66
56 http://seu.edu.bd/center_band.php?id=acacal (20099) 68
50 http://seu.edu.bd/campus_life.php?id=health_safety (16374) 69
22 http://seu.edu.bd/admission.php?id=admission_form (17658) 70
34 http://seu.edu.bd/administration.php?id=dcs (22889) 73
21 http://seu.edu.bd/admission.php?id=foreign_students (18594) 67
19 http://seu.edu.bd/admission.php?id=tuition_fee (17606) 67
25 http://seu.edu.bd/administration.php?id=the_pro_vice_chancellor_office (18486) 73
58 http://seu.edu.bd/clubs.php?id=debate_club (19736) 69
12 http://seu.edu.bd/dept/bba.php?id=overview (10189) 29
71 http://seu.edu.bd/admission/downloads/Entry_form.pdf Ignore non text/html page
95 http://seu.e

#sprank.py
Once you have a few pages in the database, you can run page rank on the pages using the sprank.py program. You simply tell it how many page rank iterations to run.

In [9]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/Coursera Learning (OWN)/Python Retrieving Processing Visualizing Data/spider.sqlite')
cur = conn.cursor()

# Find the ids that send out page rank - we only are interested
# in pages in the SCC that have in and out links
cur.execute('''SELECT DISTINCT from_id FROM Links''')
from_ids = list()
for row in cur: 
    from_ids.append(row[0])

# Find the ids that receive page rank 
to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)

# Get latest page ranks for strongly connected component
prev_ranks = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?''', (node, ))
    row = cur.fetchone()
    prev_ranks[node] = row[0]

sval = input('How many iterations:')
many = 1
if ( len(sval) > 0 ) : many = int(sval)

# Sanity check
if len(prev_ranks) < 1 : 
    print("Nothing to page rank.  Check data.")
    quit()

# Lets do Page Rank in memory so it is really fast
for i in range(many):
    # print prev_ranks.items()[:5]
    next_ranks = dict();
    total = 0.0
    for (node, old_rank) in list(prev_ranks.items()):
        total = total + old_rank
        next_ranks[node] = 0.0
    # print total

    # Find the number of outbound links and sent the page rank down each
    for (node, old_rank) in list(prev_ranks.items()):
        # print node, old_rank
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node : continue
           #  print '   ',from_id,to_id

            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if ( len(give_ids) < 1 ) : continue
        amount = old_rank / len(give_ids)
        # print node, old_rank,amount, give_ids
    
        for id in give_ids:
            next_ranks[id] = next_ranks[id] + amount
    
    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank
    evap = (total - newtot) / len(next_ranks)

    # print newtot, evap
    for node in next_ranks:
        next_ranks[node] = next_ranks[node] + evap

    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank

    # Compute the per-page average change from old rank to new rank
    # As indication of convergence of the algorithm
    totdiff = 0
    for (node, old_rank) in list(prev_ranks.items()):
        new_rank = next_ranks[node]
        diff = abs(old_rank-new_rank)
        totdiff = totdiff + diff

    avediff = totdiff / len(prev_ranks)
    print(i+1, avediff)

    # rotate
    prev_ranks = next_ranks

# Put the final ranks back into the database
print(list(next_ranks.items())[:5])
cur.execute('''UPDATE Pages SET old_rank=new_rank''')
for (id, new_rank) in list(next_ranks.items()) :
    cur.execute('''UPDATE Pages SET new_rank=? WHERE id=?''', (new_rank, id))
conn.commit()
conn.close()



How many iterations:50
1 0.3637272827030791
2 0.06646279829773083
3 0.06235835718227072
4 0.05869393911749885
5 0.055089194160974624
6 0.0516862895125577
7 0.04849091573070124
8 0.04549271456422716
9 0.04267983997978439
10 0.04004088168328439
11 0.03756509306478826
12 0.03524238607232345
13 0.0330632955741384
14 0.03101894156295168
15 0.029100993078991305
16 0.02730163427604185
17 0.025613532573243822
18 0.024029808774352682
19 0.022544009034315474
20 0.021150078559165977
21 0.019842336932087125
22 0.018615454965098904
23 0.017464432982046925
24 0.016384580444380622
25 0.015371496836705136
26 0.014421053734205862
27 0.013529377978874288
28 0.012692835895970617
29 0.011908018486408167
30 0.01117172753471542
31 0.010480962575962925
32 0.009832908668545895
33 0.009224924922993948
34 0.008654533740061249
35 0.008119410714244612
36 0.00761737516157932
37 0.007146381233116597
38 0.006704509577870385
39 0.006289959521254285
40 0.005901041727139604
41 0.0055361713136270405
42 0.005193861394483

#spdump.py
If you want to dump the contents of the spider.sqlite file, you can run spdump.py as follows

In [10]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/Coursera Learning (OWN)/Python Retrieving Processing Visualizing Data/spider.sqlite')
cur = conn.cursor()

cur.execute('''SELECT COUNT(from_id) AS inbound, old_rank, new_rank, id, url 
     FROM Pages JOIN Links ON Pages.id = Links.to_id
     WHERE html IS NOT NULL
     GROUP BY id ORDER BY inbound DESC''')

count = 0
for row in cur :
    if count < 50 : print(row)
    count = count + 1
print(count, 'rows.')
conn.close()

(1424, 1.0, -3.1579677144893343e-16, 65, 'http://seu.edu.bd/news_and_events.php')
(194, 1.0, -3.1579677144893343e-16, 67, 'http://seu.edu.bd/notice_board.php')
(109, 1.0, 0.06063936859855249, 3, 'http://seu.edu.bd/career/index.php')
(70, 1.0, 3.8132305929671046, 41, 'http://seu.edu.bd/seujse/index.php')
(64, 1.0, 0.060870615157515545, 23, 'http://seu.edu.bd/administration.php?id=seut_office')
(64, 1.0, 0.060870615157515545, 25, 'http://seu.edu.bd/administration.php?id=the_pro_vice_chancellor_office')
(64, 1.0, 0.060870615157515545, 26, 'http://seu.edu.bd/administration.php?id=registrar_office')
(64, 1.0, 0.060870615157515545, 27, 'http://seu.edu.bd/administration.php?id=accounts_and_finance_controller_office')
(64, 1.0, 0.060870615157515545, 29, 'http://seu.edu.bd/administration.php?id=library_office')
(64, 1.0, 0.060870615157515545, 31, 'http://seu.edu.bd/administration.php?id=it_administration_office')
(64, 1.0, 0.060870615157515545, 34, 'http://seu.edu.bd/administration.php?id=dcs')

#spjson.py


In [11]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/Coursera Learning (OWN)/Python Retrieving Processing Visualizing Data/spider.sqlite')
cur = conn.cursor()

print("Creating JSON output on spider.js...")
howmany = int(input("How many nodes? "))

cur.execute('''SELECT COUNT(from_id) AS inbound, old_rank, new_rank, id, url 
    FROM Pages JOIN Links ON Pages.id = Links.to_id
    WHERE html IS NOT NULL AND ERROR IS NULL
    GROUP BY id ORDER BY id,inbound''')

fhand = open('/content/drive/My Drive/Coursera Learning (OWN)/Python Retrieving Processing Visualizing Data/spider1.js','w')
nodes = list()
maxrank = None
minrank = None
for row in cur :
    nodes.append(row)
    rank = row[2]
    if maxrank is None or maxrank < rank: maxrank = rank
    if minrank is None or minrank > rank : minrank = rank
    if len(nodes) > howmany : break

if maxrank == minrank or maxrank is None or minrank is None:
    print("Error - please run sprank.py to compute page rank")
    quit()

fhand.write('spiderJson = {"nodes":[\n')
count = 0
map = dict()
ranks = dict()
for row in nodes :
    if count > 0 : fhand.write(',\n')
    # print row
    rank = row[2]
    rank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
    fhand.write('{'+'"weight":'+str(row[0])+',"rank":'+str(rank)+',')
    fhand.write(' "id":'+str(row[3])+', "url":"'+row[4]+'"}')
    map[row[3]] = count
    ranks[row[3]] = rank
    count = count + 1
fhand.write('],\n')

cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
fhand.write('"links":[\n')

count = 0
for row in cur :
    # print row
    if row[0] not in map or row[1] not in map : continue
    if count > 0 : fhand.write(',\n')
    rank = ranks[row[0]]
    srank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
    fhand.write('{"source":'+str(map[row[0]])+',"target":'+str(map[row[1]])+',"value":3}')
    count = count + 1
fhand.write(']};')
fhand.close()
conn.close()

print("Open force.html in a browser to view the visualization")


Creating JSON output on spider.js...
How many nodes? 25
Open force.html in a browser to view the visualization
